In [4]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# 1. 准备数据 (Input + Output)
# 训练数据：告诉机器什么是“标题党”，什么是“严肃新闻”
train_titles = ["震惊！这个秘密只有1%的人知道", "某市发布最新GDP数据", "惊爆眼球的真相", "财政部召开新闻发布会"]
train_labels = ["标题党", "严肃新闻", "标题党", "严肃新闻"]

# 2. 数据转换 (文字 -> 机器能懂的数字)
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_titles)

# 3. 训练模型 (Learning)
# 机器在这里寻找词汇与标签之间的概率关系（例如：“震惊”常对应“标题党”）
clf = MultinomialNB()
clf.fit(X_train, train_labels)

# 4. 预测 (Prediction)
new_title = ["大多数人都不知道的10个豆包技巧！"]
X_new = vectorizer.transform(new_title)
print(f"输入: {new_title[0]}")
print(f"预测结果: {clf.predict(X_new)[0]}")

输入: 大多数人都不知道的10个豆包技巧！
预测结果: 严肃新闻


In [6]:
from sklearn.cluster import KMeans
import numpy as np

# 1. 准备数据 (只有 Input，没有标签)
# 假设数据代表：[平均阅读时长(分钟), 平均点赞率(%)]
# 前两个用户是“快餐式用户”，后两个是“深度用户”，但机器不知道
user_behavior = np.array([
    [1, 5],   # 用户A: 读1分钟, 点赞率5%
    [2, 5],   # 用户B: 读2分钟, 点赞率5%
    [10, 80], # 用户C: 读10分钟, 点赞率80%
    [12, 90],  # 用户D: 读12分钟, 点赞率90%
    [3, 15]  # 用户D: 读12分钟, 点赞率90%
])

# 2. 训练模型 (自动发现结构)
# 我们告诉机器：请把这些人分成 2 类
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(user_behavior)

# 3. 查看分组结果
print(f"分组结果: {kmeans.labels_}")
# 输出可能为 [1 1 0 0] 或 [0 0 1 1]，表示前两人一组，后两人一组

分组结果: [1 1 0 0 1]


In [7]:
import random

# 场景：系统有两个标题版本，不知道哪个点击率更高
titles = ["标题A (平庸)", "标题B (爆款)"]
# 上帝视角的真实点击率（机器一开始不知道）
true_ctr = {"标题A (平庸)": 0.1, "标题B (爆款)": 0.8}

# 机器的策略：Epsilon-Greedy (探索 vs 利用)
def choose_title(epsilon=0.2):
    # 20% 的概率“探索”：随机瞎蒙，为了发现新机会
    if random.random() < epsilon:
        return random.choice(titles)
    # 80% 的概率“利用”：选当前认为最好的（此处简化为假设机器已经觉得B好）
    else:
        return "标题B (爆款)"

# 模拟一次推荐过程
selected_title = choose_title()
# 模拟用户反馈：如果随机数小于点击率，则算点击（Reward=1）
user_clicked = 1 if random.random() < true_ctr[selected_title] else 0

print(f"机器选择: {selected_title}")
print(f"用户反馈: {'点击' if user_clicked else '滑走'}")
# 强化学习的核心：机器会根据这个反馈更新对标题价值的判断

机器选择: 标题B (爆款)
用户反馈: 滑走


In [9]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 1. 模拟用户阅读记录 (0=没读过, 1=读过)
# 我们有4位用户，User_A 是我们要推荐的目标
data = {
    'Article_Tech': [1, 1, 0, 0],  # 科技文章
    'Article_AI':   [1, 1, 0, 0],  # AI文章
    'Article_Star': [0, 0, 1, 1],  # 娱乐文章
    'Article_Movie': [0, 0, 1, 1]   # 电影文章
}
users = ['User_A', 'User_B', 'User_C', 'User_D']
df = pd.DataFrame(data, index=users)

# 2. 计算用户相似度 (数学魔法：余弦相似度)
# 机器计算谁跟 User_A 的阅读口味最像
user_similarity = cosine_similarity(df)
user_sim_df = pd.DataFrame(user_similarity, index=users, columns=users)

# 3. 找到最相似的“邻居”
target_user = 'User_A'
# 排序，找到相似度最高的（排除自己）
similar_users = user_sim_df[target_user].sort_values(ascending=False)

print("用户阅读矩阵 (机器眼中的世界):")
print(df)
print(f"\n谁跟 {target_user} 最像？")
print(similar_users[1:2]) 
# 结果应该是 User_B (相似度1.0)，因为他们读的一模一样
# 系统随后会把 User_B 读过但 User_A 没读过的文章推荐给 User_A

用户阅读矩阵 (机器眼中的世界):
        Article_Tech  Article_AI  Article_Star  Article_Movie
User_A             1           1             0              0
User_B             1           1             0              0
User_C             0           0             1              1
User_D             0           0             1              1

谁跟 User_A 最像？
User_B    1.0
Name: User_A, dtype: float64
